In [ ]:
# Import dependencies
import pandas as pd
import pymongo
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine


In [ ]:
counties_df = pd.read_excel('list-counties-us.xlsx')
counties_df.head()

In [ ]:
states_abbr = pd.read_excel('states abbrev.xlsx')
states_abbr.head()

In [ ]:
merged_df = pd.merge(counties_df, states_abbr, left_on='State or district', right_on='State', how='left')
merged_df.head()

In [ ]:
# think about the BEST way to prime this data
# is it strip county and append the word 'county?'
# do you want to filter out anything that does not contain county?

merged_df['search_term'] = merged_df['County or equivalent'].str.replace(' ', '-').str.lower()
merged_df.head()
merged_df['search_term'] = merged_df['search_term'] +  '-' + merged_df['Abbreviation'].str.lower()
merged_df['search_term'].head()

In [ ]:
merged_df.head()

In [ ]:
search_terms = list(merged_df['search_term'])
search_terms

In [ ]:
# URL of page to be scraped  
reviews_list = []

#list of counties
counties = search_terms [:3] #[ 'autauga-county-al', 'baldwin-county-al']
print(counties)
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# for county in counties:

    
#     url = f'https://www.niche.com/places-to-live/search/best-places-to-live/c/{county}/'
#     print(url)
    
   
#     #------splinter
#     browser.visit(url)
#     time.sleep(3) # Sleep for 3 seconds
#     html = browser.html
#     #Create BeautifulSoup object; parse with 'html.parser'
#     review_soup = BeautifulSoup(html, 'html.parser')
    
#     #########################
#     # workaround for the human verification stuff
#     try:
#         human_verify = review_soup.find('div', class_="page-title").find('h1').text
#         while human_verify == 'Please verify you are a human':

#             time.sleep(30)

#             review_soup = BeautifulSoup(html, 'html.parser')
#             human_verify = review_soup.find('div', class_="page-title").find('h1').text
#     except:
#         pass
#     #########################
    
#     #BeautifulSoup scrape
#     #    Postgres on AWS
#     try:
#         comment = ''
#         date   = ''
#         town  = ''
#         county_long  = county 
#         state = county.split("-")[2].upper()
#         county = county.split('-')[0].capitalize()
#     except:
#         pass
    
             
#                 # Error handling
            
#     try:
#         comment_container = review_soup.find('p', class_="search-result-feature")
#         # Identify and return comments, date, place, county of reviews
#         comment = comment_container.find('span').text
        
#        # date = review_soup.find('li', class_="review-tagline__item").text
        
#         town = review_soup.find('h2', class_="search-result__title").text
        
#        # county = review_soup.find('div', class_="global-nav__input-wrap__sherlock-overlay").text
            
#     except Exception as e:
#         print(e)
        
#     review_dict = {
#         'comment': comment,
#         'date': date,
#         'town': town,
#        # 'county': county,
#         'state': state,
#         'county_long': county_long
#     }
        
#     reviews_list.append(review_dict)
    

# reviews_list    
# browser.quit()

In [ ]:
reviews_list = []
for county in counties:

    
    url = f'https://www.niche.com/places-to-live/search/best-places-to-live/c/{county}/'
    print(url)
   
   
    #------splinter
    browser.visit(url)    
    time.sleep(3) # Sleep for 3 seconds
    html = browser.html
    #Create BeautifulSoup object; parse with 'html.parser'
    review_soup = BeautifulSoup(html, 'html.parser')
    
    #########################
    # workaround for the human verification stuff
    try:
        human_verify = review_soup.find('div', class_="page-title").find('h1').text
        while human_verify == 'Please verify you are a human':

            time.sleep(30)

            review_soup = BeautifulSoup(html, 'html.parser')
            human_verify = review_soup.find('div', class_="page-title").find('h1').text
    except:
        pass
    #########################
    
    #BeautifulSoup scrape
    #    Postgres on AWS
    county_long  = county 
    state = county.split("-")[2].upper()
    county = county.split('-')[0].capitalize()
    commentssoup = []
             
                # Error handling
            
    try:
        comment_containers = review_soup.find_all('p', class_="search-result-feature")
        print(len(comment_containers))
        for container in comment_containers:
            print(container)
            comment = ''
            date   = ''
            town  = ''

           
            # Identify and return comments, date, place, county of reviews
            comment = container.find('span').text
            #reviews.append(comment)
        
            # date = review_soup.find('li', class_="review-tagline__item").text
        
            town = review_soup.find('h2', class_="search-result__title").text
        
            # county = review_soup.find('div', class_="global-nav__input-wrap__sherlock-overlay").text
            
            review_dict = {
            'comment': comment,
            #'date': date,
            'town': town,
            # 'county': county,
            'state': state,
            'county_long': county_long
            }

            reviews_list.append(review_dict)
            print(reviews_list)
            
    except Exception as e:
        print(e)


reviews_list    
browser.quit()

In [ ]:
reviews_list
reviews_df = pd.DataFrame(reviews_list)
reviews_df.head(20)
len(reviews_list)

In [ ]:
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd
import mysql.connector

In [ ]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [ ]:
reviews_df.to_sql(name='review_webscraping', con=engine, if_exists = 'replace', index=True)
